# INITIALIZATION

In [1]:
import numpy as np
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tmp/data/", one_hot=True)

Extracting ./tmp/data/train-images-idx3-ubyte.gz
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [3]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [5]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [6]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Save and Restore
saver = tf.train.Saver()

# SAVE VARIABLES

In [8]:
# Set value loading operations
def param_value_load_ops(param_tensor_list):
    """

    Args:
        param_tensor_list: list of trainable parameters

    Returns:
        load_ops_list: list of ops
        load_ph_list: list of placeholders
    """
    load_ops_list = []
    load_ph_list = []

    # For each parameter
    for param_tensor in param_tensor_list:
        # Make a copy of placeholder and load it
        param_load_ph = tf.placeholder(dtype=param_tensor.dtype,
                                       shape=param_tensor.shape)
        load_ph_list.append(param_load_ph)
        # Load ops with placeholder assigned to it
        load_ops_list.append(param_tensor.assign(param_load_ph))

    return load_ops_list, load_ph_list

In [20]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    
    # TODO: save initial weights and biases (using pickle?)
    # 이거 하면 w_init, b_init 은 dict 로 나옴
    w_init = sess.run(weights)
    b_init = sess.run(biases)
    init_param_value_list = []
    # dict 에서 value 들만 모음, 순서 정확히 맞추기 위해서 key 사용
    for key in ['h1', 'h2', 'out']:
        init_param_value_list.append(w_init[key])
    for key in ['b1', 'b2', 'out']:
        init_param_value_list.append(b_init[key])
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            save_path = saver.save(sess, "./tmp/model.ckpt")
            print("Model saved in file: %s" % save_path)

    print("Optimization Finished!")

    # TODO: save trained weights and biases (using pickle)
    w_trnd = sess.run(weights)
    b_trnd = sess.run(biases)
    trnd_param_value_list = []
    for key in ['h1', 'h2', 'out']:
        trnd_param_value_list.append(w_trnd[key])
    for key in ['b1', 'b2', 'out']:
        trnd_param_value_list.append(b_trnd[key])

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))
    
    # TODO: plot Accuracy for linear combiantion of initialized/trained weights and biases

{'h2': array([[-0.27762404,  0.22762072,  0.38757521, ...,  2.06988168,
        -0.29876626, -0.57448071],
       [ 1.52011883,  2.19286799, -0.63541096, ...,  0.30030993,
         0.80979788,  0.74434966],
       [ 0.76792687, -0.24144891, -1.79406917, ...,  0.99619716,
        -0.18454124, -1.65317738],
       ..., 
       [-0.20039764,  0.21085291, -0.74506176, ...,  2.27535486,
        -1.98925662, -0.64700544],
       [-0.08790147, -0.40243375,  0.5392105 , ...,  0.35589457,
         1.08084631, -1.324682  ],
       [-1.05532098,  0.78263783, -0.06058372, ..., -1.44948232,
        -1.22827256, -1.24522638]], dtype=float32), 'h1': array([[-0.95857173, -0.10210408,  1.55283499, ..., -0.50571275,
         0.64662105,  0.3820546 ],
       [ 0.48207128,  0.10143144,  0.58966959, ...,  0.59899735,
        -0.06491955,  0.43194366],
       [ 1.26625574,  0.24863462,  0.73068857, ..., -1.29431379,
        -0.45420745, -2.41340494],
       ..., 
       [ 1.82461393,  1.10235667,  0.3688717

# RESTORE VARIABLES

In [10]:
# Get list of trainable parameters
# 이렇게 하면 default graph 상에 있는 Varialble 들의 pointer 를 꺼내는 셈
param_tensor_list = tf.trainable_variables()

In [16]:
# Build load operation
# 사용하기 쉽게 list 로 정리해주는 것이긴 한데,
# 이렇게 해도 결국은 pointer 로 사용하겠다는 것임
load_ops_list, load_ph_list = param_value_load_ops(param_tensor_list)

# DO LINEAR COMBINATION AND EVALUATE

In [22]:
with tf.Session() as sess:
    sess.run(init)
    
    # linearly combine init and trained parameters
    alphas = np.arange(0.0,2.01,.05)
    errors = []
    for alpha in alphas:
        feed_dict = {}
        for load_ph, init_value, trnd_value in zip(load_ph_list, init_param_value_list, trnd_param_value_list):
            # linearly combine values
            # 이해: 이 부분이 문법이 이해가 안 됨
            feed_dict[load_ph] = init_value*(1.-alpha) + trnd_value*alpha

        # 이해: 이 부분이 문법이 이해가 안 됨
        sess.run(load_ops_list,
                 feed_dict=feed_dict)

        # do eval by using the graph
        # Calculate accuracy for MNIST test images
        acc = sess.run(accuracy, feed_dict={X: mnist.test.images,
                                            Y: mnist.test.labels})
        print("Testing Accuracy:", acc )
        errors.append((1-acc)*100)

[[-0.95857173 -0.10210408  1.55283499 ..., -0.50571275  0.64662105
   0.3820546 ]
 [ 0.48207128  0.10143144  0.58966959 ...,  0.59899735 -0.06491955
   0.43194366]
 [ 1.26625574  0.24863462  0.73068857 ..., -1.29431379 -0.45420745
  -2.41340494]
 ..., 
 [ 1.82461393  1.10235667  0.36887178 ...,  0.23010135 -0.33743805
   1.61357749]
 [-1.03865147 -0.02187275 -0.16435048 ..., -0.93060857 -0.83678401
   0.55806476]
 [ 1.14415073 -1.17981172  0.0370765  ..., -0.08793779 -0.16978824
   0.26974216]]
[[-0.27762404  0.22762072  0.38757521 ...,  2.06988168 -0.29876626
  -0.57448071]
 [ 1.52011883  2.19286799 -0.63541096 ...,  0.30030993  0.80979788
   0.74434966]
 [ 0.76792687 -0.24144891 -1.79406917 ...,  0.99619716 -0.18454124
  -1.65317738]
 ..., 
 [-0.20039764  0.21085291 -0.74506176 ...,  2.27535486 -1.98925662
  -0.64700544]
 [-0.08790147 -0.40243375  0.5392105  ...,  0.35589457  1.08084631
  -1.324682  ]
 [-1.05532098  0.78263783 -0.06058372 ..., -1.44948232 -1.22827256
  -1.24522638]]


('Testing Accuracy:', 0.1646)
[[-0.95857167 -0.10210408  1.55283487 ..., -0.50571275  0.64662105
   0.38205457]
 [ 0.48207128  0.10143144  0.58966953 ...,  0.59899735 -0.06491955
   0.43194363]
 [ 1.26625562  0.24863462  0.73068857 ..., -1.29431379 -0.45420742
  -2.4134047 ]
 ..., 
 [ 1.82461393  1.10235667  0.36887175 ...,  0.23010135 -0.33743805
   1.61357749]
 [-1.03865147 -0.02187275 -0.16435046 ..., -0.93060857 -0.83678401
   0.5580647 ]
 [ 1.14415073 -1.17981172  0.0370765  ..., -0.08793779 -0.16978824
   0.26974216]]
[[-0.24816966  0.23654592  0.35863021 ...,  2.07531857 -0.23603089
  -0.55238318]
 [ 1.5181967   2.17908192 -0.6711973  ...,  0.27268523  0.76583421
   0.69792265]
 [ 0.69604766 -0.20288798 -1.75571942 ...,  1.03437233 -0.18711084
  -1.61859453]
 ..., 
 [-0.14004409  0.20402221 -0.73241633 ...,  2.24162245 -1.95044804
  -0.61822617]
 [-0.04271188 -0.38360003  0.54169393 ...,  0.3084498   1.01488984
  -1.25583494]
 [-0.99240363  0.77738124 -0.06189025 ..., -1.5090922

('Testing Accuracy:', 0.30419999)
[[-0.95857179 -0.10210408  1.55283511 ..., -0.50571275  0.64662105
   0.38205463]
 [ 0.48207128  0.10143145  0.58966959 ...,  0.59899735 -0.06491955
   0.43194368]
 [ 1.26625586  0.24863462  0.73068857 ..., -1.29431391 -0.45420745
  -2.41340494]
 ..., 
 [ 1.82461405  1.10235667  0.36887181 ...,  0.23010135 -0.33743805
   1.61357749]
 [-1.03865147 -0.02187275 -0.16435048 ..., -0.93060857 -0.83678401
   0.55806482]
 [ 1.14415073 -1.17981184  0.0370765  ..., -0.08793779 -0.16978824
   0.26974216]]
[[-0.23344249  0.24100852  0.34415776 ...,  2.07803726 -0.20466319
  -0.54133451]
 [ 1.51723564  2.172189   -0.68909049 ...,  0.2588729   0.74385244
   0.6747092 ]
 [ 0.66010809 -0.18360753 -1.73654473 ...,  1.05345988 -0.18839565
  -1.60130322]
 ..., 
 [-0.10986732  0.20060687 -0.72609365 ...,  2.22475648 -1.93104386
  -0.6038366 ]
 [-0.0201171  -0.37418321  0.54293567 ...,  0.28472745  0.98191172
  -1.22141159]
 [-0.96094501  0.77475303 -0.06254353 ..., -1.538

('Testing Accuracy:', 0.56400001)
[[-0.95857173 -0.10210408  1.55283499 ..., -0.50571275  0.64662105
   0.3820546 ]
 [ 0.48207128  0.10143144  0.58966959 ...,  0.59899735 -0.06491955
   0.43194366]
 [ 1.26625574  0.24863462  0.73068857 ..., -1.29431379 -0.45420745
  -2.41340494]
 ..., 
 [ 1.82461393  1.10235667  0.36887178 ...,  0.23010135 -0.33743805
   1.61357749]
 [-1.03865147 -0.02187275 -0.16435048 ..., -0.93060857 -0.83678401
   0.55806476]
 [ 1.14415073 -1.17981172  0.0370765  ..., -0.08793779 -0.16978824
   0.26974216]]
[[-0.20398813  0.2499337   0.31521279 ...,  2.0834744  -0.14192784
  -0.51923698]
 [ 1.51531339  2.15840292 -0.72487682 ...,  0.2312482   0.69988883
   0.62828219]
 [ 0.58822888 -0.14504662 -1.69819498 ...,  1.09163499 -0.19096527
  -1.56672049]
 ..., 
 [-0.04951377  0.19377618 -0.71344817 ...,  2.19102407 -1.89223516
  -0.57505727]
 [ 0.02507248 -0.35534948  0.5454191  ...,  0.23728269  0.91595525
  -1.15256441]
 [-0.89802766  0.76949644 -0.06385005 ..., -1.598

('Testing Accuracy:', 0.71469998)
[[-0.95857167 -0.10210408  1.55283499 ..., -0.50571275  0.64662105
   0.38205457]
 [ 0.48207128  0.10143144  0.58966959 ...,  0.59899735 -0.06491955
   0.43194366]
 [ 1.26625562  0.24863461  0.73068857 ..., -1.29431379 -0.45420742
  -2.41340494]
 ..., 
 [ 1.82461393  1.10235667  0.36887175 ...,  0.23010135 -0.33743805
   1.61357749]
 [-1.03865147 -0.02187275 -0.16435048 ..., -0.93060851 -0.83678401
   0.5580647 ]
 [ 1.14415073 -1.17981172  0.0370765  ..., -0.08793779 -0.16978824
   0.26974216]]
[[-0.17453375  0.25885889  0.28626782 ...,  2.08891106 -0.07919245
  -0.49713948]
 [ 1.51339126  2.14461708 -0.76066315 ...,  0.2036235   0.65592515
   0.58185518]
 [ 0.51634967 -0.10648568 -1.65984535 ...,  1.12981009 -0.19353485
  -1.53213763]
 ..., 
 [ 0.01083979  0.18694548 -0.7008028  ...,  2.15729165 -1.85342669
  -0.546278  ]
 [ 0.07026204 -0.33651575  0.54790258 ...,  0.18983793  0.84999877
  -1.08371735]
 [-0.83511031  0.76423985 -0.06515658 ..., -1.658

...

('Testing Accuracy:', 0.2156)
[[-0.95857173 -0.10210408  1.55283499 ..., -0.50571275  0.64662099
   0.38205457]
 [ 0.48207128  0.10143145  0.58966959 ...,  0.59899735 -0.06491956
   0.43194368]
 [ 1.26625562  0.24863462  0.73068857 ..., -1.29431379 -0.45420745
  -2.41340494]
 ..., 
 [ 1.82461393  1.10235667  0.36887175 ...,  0.23010135 -0.33743805
   1.61357749]
 [-1.03865147 -0.02187275 -0.16435046 ..., -0.93060857 -0.83678395
   0.55806476]
 [ 1.14415073 -1.17981172  0.0370765  ..., -0.08793779 -0.16978824
   0.26974216]]
[[ 0.20837301  0.37488645 -0.09001684 ...,  2.15959287  0.73636734
  -0.2098721 ]
 [ 1.48840308  1.96539938 -1.22588563 ..., -0.15549746  0.08439803
  -0.02169573]
 [-0.41807973  0.39480627 -1.16129959 ...,  1.62608695 -0.22693978
  -1.08256137]
 ..., 
 [ 0.79543591  0.09814653 -0.53641236 ...,  1.71877158 -1.34891546
  -0.17214757]
 [ 0.65772653 -0.09167758  0.58018744 ..., -0.42694372 -0.00743461
  -0.18870622]
 [-0.01718509  0.69590461 -0.0821415  ..., -2.4330456

('Testing Accuracy:', 0.23649999)
[[-0.95857167 -0.10210408  1.55283487 ..., -0.50571275  0.64662099
   0.3820546 ]
 [ 0.48207125  0.10143144  0.58966959 ...,  0.59899741 -0.06491955
   0.43194366]
 [ 1.26625586  0.24863464  0.73068857 ..., -1.29431391 -0.45420745
  -2.41340494]
 ..., 
 [ 1.82461405  1.10235667  0.36887181 ...,  0.23010135 -0.33743805
   1.61357737]
 [-1.03865147 -0.02187275 -0.16435048 ..., -0.93060851 -0.83678401
   0.55806476]
 [ 1.14415073 -1.17981172  0.0370765  ..., -0.08793779 -0.16978824
   0.26974216]]
[[ 0.23782738  0.38381162 -0.11896183 ...,  2.16502976  0.79910266
  -0.1877746 ]
 [ 1.48648083  1.95161343 -1.26167202 ..., -0.18312219  0.04043442
  -0.06812271]
 [-0.48995891  0.43336722 -1.12294996 ...,  1.66426229 -0.2295094
  -1.04797852]
 ..., 
 [ 0.85578948  0.09131585 -0.52376699 ...,  1.68503916 -1.31010699
  -0.14336827]
 [ 0.70291609 -0.07284388  0.58267093 ..., -0.47438848 -0.07339108
  -0.11985916]
 [ 0.04573226  0.69064814 -0.08344803 ..., -2.4926

('Testing Accuracy:', 0.2572)
[[-0.95857173 -0.10210408  1.55283499 ..., -0.50571275  0.64662111
   0.38205457]
 [ 0.48207128  0.10143144  0.58966959 ...,  0.59899735 -0.06491955
   0.43194363]
 [ 1.26625574  0.24863462  0.73068857 ..., -1.29431367 -0.45420748
  -2.41340518]
 ..., 
 [ 1.82461381  1.10235667  0.36887178 ...,  0.23010135 -0.33743802
   1.61357749]
 [-1.03865147 -0.02187275 -0.16435048 ..., -0.93060863 -0.83678401
   0.5580647 ]
 [ 1.14415073 -1.17981172  0.0370765  ..., -0.08793778 -0.16978824
   0.26974213]]
[[ 0.26728174  0.39273679 -0.14790678 ...,  2.1704669   0.8618381
  -0.16567713]
 [ 1.4845587   1.93782735 -1.29745841 ..., -0.21074687 -0.00352919
  -0.11454976]
 [-0.56183815  0.47192815 -1.08460021 ...,  1.7024374  -0.232079
  -1.01339567]
 ..., 
 [ 0.91614306  0.08448517 -0.51112157 ...,  1.65130699 -1.27129841
  -0.11458904]
 [ 0.7481057  -0.05401015  0.58515429 ..., -0.52183324 -0.13934755
  -0.05101204]
 [ 0.10864961  0.68539155 -0.08475457 ..., -2.55226564 -

('Testing Accuracy:', 0.2755)
[[-0.95857173 -0.10210408  1.55283499 ..., -0.50571275  0.64662111
   0.38205463]
 [ 0.48207134  0.10143145  0.58966964 ...,  0.59899747 -0.06491955
   0.43194368]
 [ 1.26625574  0.24863465  0.73068857 ..., -1.29431391 -0.45420751
  -2.41340494]
 ..., 
 [ 1.82461405  1.10235667  0.36887178 ...,  0.23010135 -0.33743808
   1.61357772]
 [-1.03865147 -0.02187275 -0.16435049 ..., -0.93060863 -0.83678406
   0.55806482]
 [ 1.14415073 -1.17981184  0.0370765  ..., -0.08793779 -0.16978826
   0.26974216]]
[[ 0.29673609  0.40166199 -0.17685176 ...,  2.17590404  0.92457342
  -0.1435796 ]
 [ 1.48263657  1.92404151 -1.3332448  ..., -0.23837154 -0.0474928
  -0.16097671]
 [-0.6337173   0.51048905 -1.04625058 ...,  1.74061275 -0.23464863
  -0.97881293]
 ..., 
 [ 0.97649664  0.07765448 -0.49847609 ...,  1.61757493 -1.23248994
  -0.08580983]
 [ 0.79329532 -0.03517649  0.5876379  ..., -0.569278   -0.20530391
   0.01783478]
 [ 0.17156684  0.68013501 -0.0860611  ..., -2.61187553

('Testing Accuracy:', 0.29359999)


# PLOT

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
plt.plot(alphas, errors)

plt.xlabel('alpha')
plt.ylabel('error(%)')
plt.title('The Long and Winding Road for MNIST')
plt.grid(True)
# plt.savefig("mnist_journey.png")
plt.show()

# CHECK